From https://datascience.stackexchange.com/questions/55566/tool-for-labeling-audio

In [2]:
from pathlib import Path
import pandas as pd
import librosa
import librosa.display
import numpy as np
from tqdm import tqdm


In [4]:
# build dataset pandas df
train_dataset = pd.read_pickle('saved_features_train.pickle')
test_dataset = pd.read_pickle('saved_features_test.pickle')

print('Finished feature extraction from ', len(train_dataset), ' files')

Finished feature extraction from  41265  files


In [5]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
x_train = np.array(train_dataset.feature.tolist())
y_train = np.array(train_dataset.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train)) 

x_test = np.array(test_dataset.feature.tolist())
y_test = np.array(test_dataset.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
y_test = to_categorical(le.fit_transform(y_test)) 

Using TensorFlow backend.


In [15]:
import importlib
import complex2_model

importlib.reload(complex2_model)
model = complex2_model.create_model()

In [16]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1024)              41984     
_________________________________________________________________
activation_6 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
activation_7 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)             

In [18]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
saved_folder = 'model_saves'
if not os.path.isdir(saved_folder):
    os.mkdir(saved_folder)

num_epochs = 300
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath=f'{saved_folder}/weights.best.deeper2_cnn_40.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 41265 samples, validate on 9630 samples
Epoch 1/300
41265/41265 [==============================] - 13s 308us/step - loss: 0.4618 - accuracy: 0.8212 - val_loss: 0.2279 - val_accuracy: 0.9521

Epoch 00001: val_loss improved from inf to 0.22789, saving model to model_saves/weights.best.deeper2_cnn_40.hdf5
Epoch 2/300
41265/41265 [==============================] - 15s 354us/step - loss: 0.2012 - accuracy: 0.9270 - val_loss: 0.1624 - val_accuracy: 0.9738

Epoch 00002: val_loss improved from 0.22789 to 0.16242, saving model to model_saves/weights.best.deeper2_cnn_40.hdf5
Epoch 3/300
41265/41265 [==============================] - 15s 353us/step - loss: 0.1391 - accuracy: 0.9528 - val_loss: 0.1316 - val_accuracy: 0.9777

Epoch 00003: val_loss improved from 0.16242 to 0.13164, saving model to model_saves/weights.best.deeper2_cnn_40.hdf5
Epoch 4/300
41265/41265 [==============================] - 15s 354us/step - loss: 0.1129 - accuracy: 0.9625 - val_loss: 0.1334 - val_accuracy: 0.9880



In [19]:
# Evaluating the model on the training and testing set
model.load_weights("model_saves\weights.best.deeper2_cnn_40.hdf5")
score = model.evaluate(x_train, y_train, verbose=0)
print(f"Training Accuracy: {score[1]*100:.1f}%")

score = model.evaluate(x_test, y_test, verbose=0)
print(f"Testing Accuracy: {score[1]*100:.1f}%")

Training Accuracy: 99.6%
Testing Accuracy: 99.3%
